In [5]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
#  CBSE Parent data
import sys
sys.path.insert(0, 'Models/CBSEData/CBSEPreProcessing/')

from parse_cbse import *

cbse = CBSEData()

# 1 for male 0 for female


In [4]:
cbse['Gender'] = [str(elem).replace("0.0", "Female") for elem in cbse['Gender']]
cbse['Gender'] = [str(elem).replace("1.0", "Male") for elem in cbse['Gender']]

In [8]:
cbse_sample = cbse.sample(500)

In [27]:
df1.to_csv("/home/souvic/mounted/btp/ndata/samples/cbse_sample.csv")

In [11]:
def split_name(name):
    name = str(name)
    # Trim begining spaces
    name = str(name).strip()
    firstname = str(name).split(' ')[0]
    lastname = ' '.join(str(name).split(" ")[1:])
    return firstname, lastname

def split_name_df(df, col):
    first_name = []
    last_name = []
    for name in df[col]:
        f,l = split_name(name)
        first_name.append(f)
        last_name.append(l)
        
    df['first_name'] = first_name
    df['last_name'] = last_name
    return df, first_name

In [21]:
df1, _ = split_name_df(cbse_sample, 'Name')

In [26]:
df1.head()

,Name,Gender,first_name,last_name
0,nandkishore mukati,Male,nandkishore,mukati
1,ramkranti,Female,ramkranti,
2,manmohan vyas,Male,manmohan,vyas
3,suresh prasad malviya,Male,suresh,prasad malviya
4,ashok kumari devi,Female,ashok,kumari devi


In [25]:
del df1['index']
del df1['level_0']

In [23]:
df1.reset_index(inplace=True)

In [29]:
df_check  = pd.read_csv(AIEEE_2009)

In [30]:
df_check.head()

,Unnamed: 0,roll_no,name,mother_name,father_name,category,state_no,present,p1_marks,p2_marks,air_btech,air_cat_btech,state_btech,state_cat_btech,air_barch,air_cat_barch,state_barch,state_cat_barch
0,0,22500416,SUNNY LALLER,SHARDA DEVI,DALBIR SINGH LALLER,GEN,13,True,26.0,NaN,687686,414972,34288,23484,------,------,------,------
1,1,22700891,KULDEEP SHARMA,CHAMPA DEVI,MANANTRAM,GEN,14,True,14.0,NaN,810423,479425,9337,6689,------,------,------,------
2,2,21415832,TEJPARTAP SINGH,HERJEET KAUR,TARLOCHAN SINGH,GEN,28,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21402077,HARJINDER SINGH,SUKHDEV KAUR,RANJIT SINGH,GEN,34,True,87.0,NaN,164867,114788,2714,2270,------,------,------,------
4,4,20600071,THYNALA SETHU DHARANIDHAR,THYNALA LAKSHMI RAJYAM,THYNALA VIJAYA KUMAR,OBC,02,True,194.0,NaN,22354,4249,3630,928,------,------,------,------


In [8]:
#  aieee sample 

AIEEE_2009 = "Models/Data/AIEEEData/aieee_2009.csv"
AIEEE_2010 = "Models/Data/AIEEEData/aieee_2010.csv"
AIEEE_2011 = "Models/Data/AIEEEData/aieee_2011.csv"


def getParentsData(df):
  mother_df = pd.DataFrame()
  father_df = pd.DataFrame()
  mother_df['Name'] = df['mother_name']
  mother_df['Gender'] = 'FEMALE'
  father_df['Name'] = df['father_name']
  father_df['Gender'] = 'MALE'
  name_df = pd.concat([mother_df, father_df], ignore_index=True)
  return name_df

def processDf(df):
  df['Name'] = [str(name).strip() for name in df['Name']]
  df['Name'] = [name.lower() for name in df['Name']]
  return df

def addFrequency(df):
  df['female_freq'] = df['Name'].map(df[df['Gender']=='FEMALE']['Name'].value_counts())
  df['male_freq'] = df['Name'].map(df[df['Gender']=='MALE']['Name'].value_counts())
  return df

def useMajorityLabel(df):
  df['new_labels'] = 0.0
  labels = []
  for i, name in enumerate(df['Name']):
    if np.isnan(df['female_freq'][i]) or df['male_freq'][i] > df['female_freq'][i]:
      # labels.append(1) 1 for MALE
      df.at[i,'new_labels'] = 1
    else:
      # labels.append(0) 0 for FEMALE
      df.at[i,'new_labels'] = 0
  return df

def dropDuplicates(df):
  df = df.drop_duplicates(subset='Name', keep='last')
  df = df.reset_index(drop=True)
  return df

def AIEEEData():
  a9 = pd.read_csv(AIEEE_2009)
  a10 = pd.read_csv(AIEEE_2010)
  a11 = pd.read_csv(AIEEE_2011)
  a9 = processDf(getParentsData(a9))
  a10 = processDf(getParentsData(a10))
  a11 = processDf(getParentsData(a11))
  aieee = pd.concat([a9, a10, a11], ignore_index=True)
  # Use majority label
  cbse = addFrequency(aieee)
  cbse = useMajorityLabel(cbse)
  cbse = dropDuplicates(cbse)
  final_df = pd.DataFrame()
  final_df['Name'] = cbse['Name']
  final_df['Gender'] = cbse['new_labels']
  return final_df

In [32]:
aieee  = AIEEEData()

In [35]:
aieee['Gender'] = [str(elem).replace("0.0", "Female") for elem in aieee['Gender']]
aieee['Gender'] = [str(elem).replace("1.0", "Male") for elem in aieee['Gender']]

In [39]:
sam = aieee.sample(500)

In [41]:
sam.reset_index(inplace=True)

In [44]:
del sam['index']

In [45]:
sam

,Name,Gender
0,prahlad singh dangi,Male
1,sarvesh kumari,Female
2,shanichri devi,Female
3,marthala savithri,Female
4,kakali roy,Female
...,...,...
495,usha natarajan,Female
496,burugula bhadraiah,Male
497,ajay kumar srivastay,Male
498,bimala devi,Female


In [46]:
df1, _ = split_name_df(sam, 'Name')

,Name,Gender,first_name,last_name
0,prahlad singh dangi,Male,prahlad,singh dangi
1,sarvesh kumari,Female,sarvesh,kumari
2,shanichri devi,Female,shanichri,devi
3,marthala savithri,Female,marthala,savithri
4,kakali roy,Female,kakali,roy
...,...,...,...,...
495,usha natarajan,Female,usha,natarajan
496,burugula bhadraiah,Male,burugula,bhadraiah
497,ajay kumar srivastay,Male,ajay,kumar srivastay
498,bimala devi,Female,bimala,devi


In [48]:
df1.to_csv("/home/souvic/mounted/btp/ndata/samples/aieee_sample.csv")

In [1]:
CBSE_2014 = "Models/Data/CBSEData/2014_complete_data.csv"
CBSE_2015 = "Models/Data/CBSEData/2015_complete_data.csv"

In [6]:
import pandas as pd

cbse14 = pd.read_csv(CBSE_2014, dtype=object, encoding= 'unicode_escape')
cbse15 = pd.read_csv(CBSE_2015, dtype=object, encoding= 'unicode_escape')

In [7]:
cbse14.shape

(44722, 424)

In [10]:
c14 = processDf(getParentsData(cbse14))
c15 = processDf(getParentsData(cbse15))

In [13]:
c15.shape

(186038, 2)

In [22]:
c15[c15['Gender']=='FEMALE'].drop_duplicates().shape

(51744, 2)

In [21]:
c14[c14['Gender']=='FEMALE'].drop_duplicates().shape

(25779, 2)

In [20]:
c15[c15['Gender']=='MALE'].drop_duplicates().shape

(63434, 2)

In [19]:
c14[c14['Gender']=='MALE'].drop_duplicates().shape

(31573, 2)